In [1]:
# !pip install --upgrade scipy

In [1]:
'''
Authors: Daniel M. Low
License: See license in github repository
'''

import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import soundfile as sf
import os 
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit
from sklearn import metrics

In [2]:
ts = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')

In [3]:


pd.set_option("display.max_columns", None)
# pd.options.display.width = 0


on_colab = False

if on_colab:
  from google.colab import drive
  project_name = 'project_name'
  drive.mount('/content/drive')
  input_dir = f'/content/drive/MyDrive/datum/{project_name}/data/input/'
  output_dir = f'/content/drive/MyDrive/datum/{project_name}/data/output/'
else:
  input_dir = './data/input/'
  output_dir = './data/output/'

os.makedirs(output_dir, exist_ok=True)



In [4]:
audio_files = os.listdir(input_dir+'audios_16khz/')
audio_files.sort()

In [5]:
audio_files_speech = [n for n in audio_files if 'Speech_' in n]

In [14]:
input_dir_audio = output_dir+'audio_16khz_trimmed_to_3500ms/'
os.makedirs(input_dir_audio, exist_ok=True)

In [15]:
import soundfile as sf

durationSeconds = 3.5
sampling_rate = 16000
for filename in audio_files_speech:
    data, sr = librosa.load(input_dir+'audios_16khz/'+filename, sr=sampling_rate)
    trimmed = librosa.util.fix_length(data, int(sr * durationSeconds))
    sf.write(input_dir_audio+filename, trimmed,sr)
    


# Feature extraction

In [ ]:
import opensmile # audio feature extraction
feature_extraction_model = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals, #summarize statistics over time signal
)

audio_files = os.listdir(input_dir_audio)

embeddings_list = []
wav_files_all = []
for i, audio_files_i in enumerate(audio_files):
    if i%50 == 0:
        print(i)
    embeddings = feature_extraction_model.process_file(input_dir_audio+audio_files[i])
    embeddings_list.append(embeddings)
    wav_files_all.append(audio_files_i)

embeddings_df = pd.concat(embeddings_list )
embeddings_df['wav_file'] = audio_files
embeddings_df.to_csv('input/features/egemaps_trimmed_to_3500ms_reading.csv')

In [25]:
embeddings_df['sid'] = [n.split('_')[0] for n in embeddings_df['wav_file'].values]
embeddings_df['target'] = [0 if 'Norm' in n else 1 for n in embeddings_df['wav_file'].values]

embeddings_df.to_csv('./data/input/features/egemaps_trimmed_to_3500ms_reading.csv')

# confirm duration is fixed

In [43]:
import librosa

df = pd.read_csv(input_dir+'features/egemaps_trimmed_to_3500ms_reading.csv', index_col = 0)
# display(df['token'].value_counts())
print(df.shape)

duration_d = {}

files = os.listdir(input_dir+'audio_16khz_trimmed_to_3500ms/')


for sid, target, filename in df[['sid','target', 'wav_file']].values:
    
    # filename2 = filename.replace('Speech', 'Speech_') 
    
    # try: 
    duration = float(librosa.get_duration(filename=input_dir+f'audio_16khz_trimmed_to_3500ms/{filename}', sr=16000) )
    duration_d[filename] = [sid, target,duration]
    # except:
    # print(f'{filename}.wav not found')


duration_df = pd.DataFrame(duration_d, index = ['sid', 'target', 'duration']).T

# duration_df['task']  = [n[:-1] for n in duration_df['token'].values]
# display(duration_df['token'].value_counts())
# display(duration_df['task'].value_counts())
duration_df = duration_df.reset_index()
duration_df.columns = ['filename', 'sid', 'target', 'duration']
duration_df


(453, 93)


,filename,sid,target,duration
0,VFPNorm5_Speech_3.wav,VFPNorm5,0,3.5
1,VFP3_Speech_3.wav,VFP3,1,3.5
2,VFPNorm11_Speech_2.wav,VFPNorm11,0,3.5
3,VFPNorm44_Speech_2.wav,VFPNorm44,0,3.5
4,VFPNorm32_Speech_1.wav,VFPNorm32,0,3.5
...,...,...,...,...
448,VFPNorm25_Speech_2.wav,VFPNorm25,0,3.5
449,VFPNorm54_Speech_1.wav,VFPNorm54,0,3.5
450,VFP18_Speech_2.wav,VFP18,1,3.5
451,VFPNorm77_Speech_2.wav,VFPNorm77,0,3.5


In [49]:
duration_df['duration']=pd.to_numeric(duration_df['duration'])
type(duration_df['duration'].values[0])


numpy.float64

In [51]:
duration_df.groupby('target')['duration'].describe()

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
0,231.0,3.5,0.0,3.5,3.5,3.5,3.5,3.5
1,222.0,3.5,0.0,3.5,3.5,3.5,3.5,3.5


# Bootstrapping classification

In [58]:

models = [
    LogisticRegressionCV(solver='liblinear', penalty = 'l1', max_iter = 100),
    SGDClassifier(loss='log', penalty="elasticnet", early_stopping=True, max_iter = 5000),
    MLPClassifier(alpha = 1, max_iter= 1000),
    RandomForestClassifier(n_estimators= 100)
]


names = ['LogisticRegressionCV', 'SGDClassifier', "MLPClassifier","RandomForestClassifier"]

In [60]:
import get_feature_names

egemaps_variables = get_feature_names.egemaps

In [63]:
%%time

os.makedirs(output_dir+'trimmed', exist_ok=True)

toy = False

for df_name, task_type_df in zip(['speech'], [
    'egemaps_trimmed_to_3500ms_reading.csv'
    ]):
    trimmed_df = pd.read_csv(input_dir+f'features/{task_type_df}', index_col = 0)
    print(df_name, '\n====')

    for null_model in [True, False]:
        print('\npermute', null_model)
    
        if toy:
          n_splits = 3      
        else:
          n_splits = 50

        # bs = cross_validation.Bootstrap(len(y), n_bootstraps=n_bootstraps, random_state=123,n_test = 0.2) # or add source code


        X = trimmed_df[egemaps_variables].values
        y = trimmed_df['target'].values
        if null_model:
            y = np.random.permutation(y) #CHECK
        groups = trimmed_df['sid'].values

        y_pred_all = {}
        roc_auc_all = {}
        for model, name in zip(models, names):
          y_pred_all[name] = []
          roc_auc_all[name] = []
          pipe = Pipeline(steps=[
                  ('scaler', StandardScaler()), 
                  ('model', model)])


          ## Performing bootstrapping
          splitter = GroupShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=0)
          for i, (train_index, test_index) in enumerate(splitter.split(X, y, groups)):

              X_train, X_test = X[train_index], X[test_index]
              y_train, y_test = y[train_index], y[test_index]
              pipe.fit(X_train,y_train)     

              # # Evaluate
              # y_proba = pipe.predict_proba(X_test)       # Get predicted probabilities
              # y_proba_1 = y_proba[:,1]
              # y_pred = np.argmax(y_proba, axis=1) 
              # roc_auc = metrics.roc_auc_score(y_test, y_proba_1)  

              y_pred = pipe.predict(X_test) 
              roc_auc = metrics.roc_auc_score(y_test, y_pred)  # ROC AUC takes probabilities but here we match what pydra-ml does: https://github.com/nipype/pydra-ml/issues/56

              y_pred_all[name].append(y_pred)
              roc_auc_all[name].append(roc_auc)

        results_i = []
        for name in ['LogisticRegressionCV','MLPClassifier','RandomForestClassifier','SGDClassifier']:
          scores = roc_auc_all.get(name)
          roc_auc_median = np.round(np.median(scores),2)
          roc_auc_5 = np.round(np.percentile(scores, 5),2)
          roc_auc_95 = np.round(np.percentile(scores, 95),2)
          results_str = f'{roc_auc_median} ({roc_auc_5}–{roc_auc_95}; )'
          results_str = results_str.replace('0.', '.')
          results_i.append([name, results_str])
            
          if null_model:
            print(name, str(roc_auc_median).replace('0.', '.'))
        if not null_model:
            results_i_df = pd.DataFrame(results_i, ).T
            display(results_i_df)
            results_i_df.to_csv(output_dir+f'trimmed/results_trimmed_{df_name}_permute-{null_model}_{ts}.csv')

            
                




speech 
====

permute True


/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/

LogisticRegressionCV .5
MLPClassifier .48
RandomForestClassifier .5
SGDClassifier .5

permute False


/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,0,1,2,3
0,LogisticRegressionCV,MLPClassifier,RandomForestClassifier,SGDClassifier
1,.87 (.8–.91; ),.87 (.8–.94; ),.88 (.79–.95; ),.82 (.75–.89; )


CPU times: user 17min 33s, sys: 3min 43s, total: 21min 17s
Wall time: 15min 48s
